In [1]:
import pandas as pd
import numpy as np

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


### count features

In [2]:
# one way
ip = pd.read_feather('../data/data/features/count/ip.feather').astype('uint32')
app = pd.read_feather('../data/data/features/count/app.feather').astype('uint32')
os = pd.read_feather('../data/data/features/count/os.feather').astype('uint32')

# n-way
ip_day_hour = pd.read_feather('../data/data/features/count/ip_day_hour.feather').astype('uint32')
ip_app = pd.read_feather('../data/data/features/count/ip_app.feather').astype('uint32')
ip_app_os = pd.read_feather('../data/data/features/count/ip_app_os.feather').astype('uint32')
ip_device = pd.read_feather('../data/data/features/count/ip_device.feather').astype('uint32')
app_channel = pd.read_feather('../data/data/features/count/app_channel.feather').astype('uint32')
ip_hour_os = pd.read_feather('../data/data/features/count/ip_hour_os.feather').astype('uint32')
ip_hour_app = pd.read_feather('../data/data/features/count/ip_hour_app.feather').astype('uint32')

# user count
user = pd.read_feather('../data/data/features/count/user_count.feather').astype('uint32')
user_app = pd.read_feather('../data/data/features/count/user_app_count.feather').astype('uint32')

### unique features

In [3]:
ip_app_unq = pd.read_feather('../data/data/features/unique/ip_app_unq.feather').astype('uint32')
ip_channel_unq = pd.read_feather('../data/data/features/unique/ip_channel_unq.feather').astype('uint32')

### next click

In [4]:
next_click_train = pd.read_feather('../data/data/features/next_click/next_click_train.feather')
next_click_test = pd.read_feather('../data/data/features/next_click/next_click_test.feather')
next_click_train_1 = pd.read_feather('../data/data/features/next_click/next_click_train_1.feather')
next_click_test_1 = pd.read_feather('../data/data/features/next_click/next_click_test_1.feather')

### running

In [5]:
ip_enc_train = pd.read_feather('../data/data/features/running/train_ip.feather')
ip_enc_valid = pd.read_feather('../data/data/features/running/valid_ip.feather')
ip_enc_test = pd.read_feather('../data/data/features/running/test_ip.feather')

ip_app_enc_train = pd.read_feather('../data/data/features/running/train_ip_app.feather')
ip_app_enc_valid = pd.read_feather('../data/data/features/running/valid_ip_app.feather')
ip_app_enc_test = pd.read_feather('../data/data/features/running/test_ip_app.feather')

### rank

In [6]:
ip_rank = pd.read_feather('../data/data/features/rank/ip.feather')
app_channel_rank = pd.read_feather('../data/data/features/rank/app_channel.feather')
app_os_rank = pd.read_feather('../data/data/features/rank/app_os.feather')
channel_os_rank = pd.read_feather('../data/data/features/rank/channel_os.feather')

### merge

In [7]:
def reduce_size(data):
    for column in list(data.columns):
        if data[column].max() <= 250:
            data[column] = data[column].astype('uint8')
        elif data[column].max() <= 65000 and data[column].min() >= 0:
            data[column] = data[column].astype('uint16')
    return data
    
def merge(data):
    print('merge starts..', data.shape)
    # one way counts
    data = data.merge(ip, on='ip')
    data = data.merge(os, on='os')
    data = data.merge(app, on='app')
    data = reduce_size(data)
    print('one way merge complete...', data.shape)
    # n way counts
    data = data.merge(ip_day_hour, on=['ip','day','hour'])
    data = data.merge(ip_app, on=['ip','app'])
    data = data.merge(ip_app_os, on=['ip','app','os'])
    data = data.merge(ip_device, on=['ip','device'])
    data = data.merge(app_channel, on=['app','channel'])
    data = data.merge(ip_hour_os, on=['ip','hour','os'])
    data = data.merge(ip_hour_app, on=['ip','hour','app'])
    data = reduce_size(data)
    print('n way merge complete...', data.shape)
    # unique counts
    data = data.merge(ip_app_unq, on=['ip'])
    data = data.merge(ip_channel_unq, on=['ip'])
    data = reduce_size(data)
    # user counts
    data = data.merge(user, on=['ip','device','os'])
    data = data.merge(user_app, on=['ip','device','os','app'])
    data = reduce_size(data)
    print('user counts merge complete...', data.shape)
    # other features
    data = data.merge(next_click_train, on='click_id')
    data = data.merge(ip_enc_train, on='click_id', how='left')
    data = data.merge(ip_app_enc_train, on='click_id', how='left')
    data = data.fillna(0)
    data = reduce_size(data)
    print('other merge complete...', data.shape)
    data = data.reset_index(drop=True)
    return data

In [8]:
dtypes = {}
dtypes['ip'] = 'uint32'
dtypes['app'] = 'uint16'
dtypes['device'] = 'uint16'
dtypes['os'] = 'uint16'
dtypes['channel'] = 'uint16'
dtypes['is_attributed'] = 'uint8'
dtypes['click_id'] = 'uint32'
usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed']
train_data = pd.read_csv('../data/download/train.csv', dtype=dtypes, usecols=usecols)
train_data['click_time'] = pd.to_datetime(train_data['click_time'])
train_data['hour'] = train_data['click_time'].map(lambda x : x.hour)
train_data['click_id'] = train_data.index
train_data['day'] = train_data['click_time'].dt.day
train_data = train_data[['click_id','is_attributed','day','hour','ip','app','os','device','channel']]
train_data.head()

,click_id,is_attributed,day,hour,ip,app,os,device,channel
0,0,0,6,14,83230,3,13,1,379
1,1,0,6,14,17357,3,19,1,379
2,2,0,6,14,35810,3,13,1,379
3,3,0,6,14,45745,14,13,1,478
4,4,0,6,14,161007,3,13,1,379


In [9]:
train_data = reduce_size(train_data)

In [10]:
train_data = merge(train_data)
train_data.to_feather('../data/data/model/full_data.feather')

merge starts.. (184903890, 9)
one way merge complete... (184903890, 12)
n way merge complete... (184903890, 19)
user counts merge complete... (184903890, 23)
other merge complete... (184903889, 26)
